In [10]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from bpitnorm.modules.BatchPitNormalization import BatchPitNorm1d

device = "cuda" if torch.cuda.is_available() else "cpu"

data = np.genfromtxt("Data/taiwaneseBankruptcyPred.csv", delimiter=",", skip_header=1)
data = data[~(np.isnan(data)).any(axis=1)]
data = data[~(np.isinf(data)).any(axis=1)]

def data_split(data, seed):
  X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(data[:, 1:], data[:, 0], test_size=0.2, random_state=seed)
  y_train_raw = y_train_raw.reshape(-1, 1)
  y_test_raw = y_test_raw.reshape(-1, 1)

  scaler1 = StandardScaler()
  scaler_x = scaler1.fit(X_train_raw)
  X_train = scaler_x.transform(X_train_raw)
  X_test = scaler_x.transform(X_test_raw)

  scaler2 = StandardScaler()
  scaler_y = scaler2.fit(y_train_raw)
  y_train = scaler_y.transform(y_train_raw)
  y_test = scaler_y.transform(y_test_raw)

  X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
  y_train = torch.tensor(y_train, dtype=torch.float32, device=device).reshape(-1, 1)
  X_test = torch.tensor(X_test, dtype=torch.float32).to(device=device)
  y_test = torch.tensor(y_test, dtype=torch.float32, device=device).reshape(-1, 1)
  return X_train, y_train, X_test, y_test, scaler_y

In [11]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

def train_model(model, loops):
    model.to(device=device)
    acc_lst = np.zeros(loops)
    torch.backends.cudnn.deterministic = True
    seeds = range(loops)

    for i, seed in zip(range(loops), seeds):
        X_train, y_train, X_test, y_test, scaler_y = data_split(data, seed)
        torch.manual_seed(seed)
        loss_fn = nn.BCEWithLogitsLoss().to(device=device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        train_loader = DataLoader(list(zip(X_train, y_train)), batch_size=150, shuffle=True)
        n_epochs = 30
        for epoch in range(n_epochs):
            model.train()
            for x_batch, y_batch in train_loader:
                x_batch, y_batch = x_batch.to(device=device), y_batch.to(device=device)
                y_pred = model(x_batch)
                loss = loss_fn(y_pred, y_batch)
                loss.backward(retain_graph=True)
                optimizer.step()
                optimizer.zero_grad()
        model.eval()
        y_test_inverse = scaler_y.inverse_transform(y_test.cpu())
        y_pred = model(X_test, dim=1).argmax(dim=1)
        y_pred = torch.softmax(y_pred)
        y_pred = scaler_y.inverse_transform(y_pred.cpu().detach().numpy())
        acc = (np.around(y_pred) == np.around(y_test_inverse)).mean()
        print("y_pred =", y_pred)
        print("number of 1 predicted:", np.sum(np.where(np.around(y_pred)==1)))
        acc_lst[i] = float(acc)
        print("accuracy:", float(acc))
    return acc_lst

In [12]:
model1 = nn.Sequential(
    nn.Linear(data.shape[1]-1, 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 10),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(10, 1))

acc1 = train_model(model1, 10)
plt.boxplot(acc1, labels=["Model 1"])
plt.title("Accuracy")
plt.show()

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model2 = nn.Sequential(
    nn.Linear(95, 100),
    BatchPitNorm1d(num_features=100, num_pit_samples=200, take_num_samples_when_full=0, dev=device),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid())

model2.to(device=device)

r2Score2, MAE2 = train_model(model2, 5)
plt.boxplot(r2Score2, labels=["Model 2"])
plt.title("R2-score")
plt.show()
print("Mean house value from all houses: ", round(np.mean(data[:, -1]), -3))
plt.boxplot(MAE2, labels=["Model 2"])
plt.title("Mean absolute error")
plt.show()

/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q25 = torch.quantile(input=data, q=.25, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:104: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q75 = torch.quantile(input=data, q=.75, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: Us

y_pred = [[0.0318974 ]
 [0.03189734]
 [0.03189734]
 ...
 [0.03189734]
 [0.03189734]
 [0.03189734]]
number of 1 predicted: 0
accuracy: 0.966275659824047


/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q25 = torch.quantile(input=data, q=.25, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:104: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q75 = torch.quantile(input=data, q=.75, dim=0)


y_pred = [[0.03116407]
 [0.03116407]
 [0.03116408]
 ...
 [0.03116407]
 [0.03116407]
 [0.03116407]]
number of 1 predicted: 0
accuracy: 0.9633431085043989


/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q25 = torch.quantile(input=data, q=.25, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:104: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q75 = torch.quantile(input=data, q=.75, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: Us

y_pred = [[0.0335472]
 [0.0335472]
 [0.0335472]
 ...
 [0.0335472]
 [0.0335472]
 [0.0335472]]
number of 1 predicted: 0
accuracy: 0.9728739002932552


/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q25 = torch.quantile(input=data, q=.25, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:104: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q75 = torch.quantile(input=data, q=.75, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: Us

y_pred = [[0.03318057]
 [0.03318057]
 [0.03318057]
 ...
 [0.03318057]
 [0.03332671]
 [0.03318057]]
number of 1 predicted: 0
accuracy: 0.9714076246334311
y_pred = [[0.03079743]
 [0.20353791]
 [0.03079743]
 ...
 [0.03079743]
 [0.03079743]
 [0.03079743]]
number of 1 predicted: 0
accuracy: 0.9618768328445748


/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:103: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q25 = torch.quantile(input=data, q=.25, dim=0)
/home/ubuntu/batch-pit-normalization/bpitnorm/modules/BatchPitNormalization.py:104: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::quantile.scalar. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  q75 = torch.quantile(input=data, q=.75, dim=0)


ValueError: too many values to unpack (expected 2)

In [ ]:
model3 = nn.Sequential(
    BatchPitNorm1d(num_features=8, num_pit_samples=100, take_num_samples_when_full=50, dev=device),
    nn.Linear(8, 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 10),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.BatchNorm1d(10),
    # BatchPitNorm1d(num_features=10, num_pit_samples=1000, take_num_samples_when_full=50, dev=device),
    nn.Linear(10, 1))
model3.to(device=device)

r2Score3, MAE3 = train_model(model3, 10)
plt.boxplot(r2Score3, labels=["Model 3"])
plt.title("R2-score")
plt.show()
print("Mean house value from all houses: ", round(np.mean(data[:, -1]), -3))
plt.boxplot(MAE3, labels=["Model 3"])
plt.title("Mean absolute error")
plt.show()

In [ ]:
plt.boxplot([r2Score1, r2Score2, r2Score3], labels=["Model 1", "Model 2", "Model 3"])
plt.title("R2-score")
plt.show()
print("Mean house value from all houses: ", round(np.mean(data[:, -1]), -3))
plt.boxplot([MAE1, MAE2, MAE3], labels=["Model 1", "Model 2", "Model 3"])
plt.title("Mean absolute error")
plt.show()